In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# GDP

In [2]:
df_gdp_world = pd.read_csv('data/GDP_world.csv', skiprows = 4)
df_gdp_world.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.112502e+10,2.161623e+10,2.350628e+10,2.804836e+10,2.592067e+10,2.947210e+10,...,1.006526e+12,9.273485e+11,8.851764e+11,1.021043e+12,1.007196e+12,1.000834e+12,9.275933e+11,1.081998e+12,1.169484e+12,NaN


In [3]:
df_gdp_world.drop(['Unnamed: 67'], axis=1, inplace=True)
# Calculando a variação percentual a partir da quinta coluna
df_variacao = df_gdp_world.iloc[:, 4:].pct_change(axis=1, fill_method=None) * 100

# Concatenando os DataFrames
df_plot = pd.concat([df_gdp_world['Country Name'], df_variacao], axis=1)

In [4]:
# Definir aplicativo Dash
app = dash.Dash(__name__)

# Selecionar países para visualização
paises = ['Argentina', 'Brazil', 'United States']

# Layout da aplicação
app.layout = html.Div([
    html.Label('Selecione os países:'),
    dcc.Dropdown(
        id='dropdown-paises',
        options=[{'label': pais, 'value': pais} for pais in df_plot['Country Name'].unique()],
        value=paises,
        multi=True
    ),
    dcc.RadioItems(
        id='radio-gdp-type',
        options=[
            {'label': 'Variação Percentual', 'value': 'percentual'},
            {'label': 'Valor Bruto', 'value': 'bruto'}
        ],
        value='percentual',
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='line-chart')
])

# Callback para atualizar o gráfico com base nas seleções
@app.callback(
    Output('line-chart', 'figure'),
    [Input('dropdown-paises', 'value'),
     Input('radio-gdp-type', 'value')]
)
def update_figure(selected_countries, gdp_type):
    if gdp_type == 'percentual':
        df_selected = df_plot
    else:
        df_selected = df_gdp_world  # Alterado para df para exibir o valor bruto

    df_paises = df_selected[df_selected['Country Name'].isin(selected_countries)]
    df_paises_long = df_paises.melt(id_vars='Country Name', var_name='Year', value_name='Valor')

    # Remove valores 'Year' da coluna Year
    df_paises_long = df_paises_long[df_paises_long['Year'] != 'Year']
    # Converte Year para datetime
    df_paises_long['Year'] = pd.to_datetime(df_paises_long['Year'], format='%Y', errors='coerce')

    fig = px.line(df_paises_long, x='Year', y='Valor', color='Country Name',
                  title=f'{gdp_type.capitalize()} do GDP por Ano')
    fig.update_layout(xaxis_title='Ano', yaxis_title=f'{gdp_type.capitalize()} do GDP')

    return fig
    
# Executar o aplicativo Dash
if __name__ == '__main__':
    app.run_server(debug=True)

---

# Inflation